In [2]:
import shutil
shutil.copy('../GAMS/base/runRBA.gms', './runRBA.gms');
shutil.copy('../GAMS/base/soplex.opt', './soplex.opt');

In [ ]:
inputs = {'species': 'RBA_species.txt',
          'rxns': 'RBA_rxns.txt',
          'prosyn_rxns': 'RBA_prosyn.txt',
          'nuc_translation': 'RBA_nuc_translation.txt',
          'mito_translation': 'RBA_mito_translation.txt',
          'uptake': 'RBA_rxns_EXREV.txt',
          'media': 'RBA_rxns_EXREV_YNB.txt',
          'sij': 'RBA_sij.txt',
          'protein_length': 'RBA_proteinLength.txt',
          'kapp': 'RBA_kapp.txt'
          'enzyme_capacity_declares': 'RBA_enzCapacityConstraints_declares.txt',
          'enzyme_capacity_eqns': 'RBA_enzCapacityConstraints_eqns.txt',
          'kribonuc': '10.5*3600',
          'kribomito': '10.5*3600',
          'mu': '0'}

In [4]:
with open('./runRBA.gms') as f:
    gmstxt = f.read().split('\n')
gmstxt[6] = '$set mu 0.15'
with open('./runRBA.gms', 'w') as f:
    f.write('\n'.join(gmstxt))

In [ ]:
import shutil
shutil.copy('../GAMS/runRBA.gms', './runRBA.gms');
shutil.copy('../GAMS/soplex.opt', './soplex.opt');

In [ ]:
with open('./runRBA.modelStat.txt') as f:
    modelStat = f.read()
modelStat = modelStat.replace('\n', '')
modelStat = modelStat.replace(' ', '')
modelStat = int(float(modelStat))

In [ ]:
import os
# Test evaluation at zero
os.system('module load gams\n' + 'gams runRBA.gms --mu=0 o=/dev/null')
stat = get_GAMS_modelStat('./runRBA.modelStat.txt')
if stat == 'infeasible':
    print('Model is infeasible at mu = 0, check model connectivity and other bugs')

In [1]:
class bcolors:
    GREEN = '\033[92m' #GREEN
    RED = '\033[91m' #RED
    RESET = '\033[0m' #RESET COLOR

In [4]:
print(f"{bcolors.GREEN}mu = {1:.7f}, status = {'opt'}{bcolors.RESET}")

mu = 1.0000000, status = opt


In [ ]:
mu_min0 = 0.18; mu_max0 = 0.20; mu_tol = 1e-4; maxiter = 100;
mu_min = mu_min0; mu_max = mu_max0; itercount = 0;

# Start binary search
# Evaluate min feasibility
mu = mu_min;
os.system('module load gams\n' + 'gams runRBA.gms --mu=' + str(mu) + ' o=/dev/null')
stat = get_GAMS_modelStat('./runRBA.modelStat.txt')
if stat == 'infeasible':
    mu_max = mu_min
    mu_min = 0
    
# Evaluate max infeasibility
mu = mu_max;
os.system('module load gams\n' + 'gams runRBA.gms --mu=' + str(mu) + ' o=/dev/null')
stat = get_GAMS_modelStat('./runRBA.modelStat.txt')
while stat == 'optimal':
    mu_max = 1.5*mu_max
    mu = mu_max;
    os.system('module load gams\n' + 'gams runRBA.gms --mu=' + str(mu) + ' o=/dev/null')
    stat = get_GAMS_modelStat('./runRBA.modelStat.txt')

In [ ]:
# Update min-max
mu = float(mu_min + mu_max) / 2; fba_final = None;
while mu_max - mu_min > mu_tol and itercount < maxiter:
    itercount += 1
        
    os.system('module load gams\n' + 'gams runRBA.gms --mu=' + str(mu) + ' o=/dev/null')
    stat = get_GAMS_modelStat('./runRBA.modelStat.txt')
    if stat == 'optimal':
        mu_min = mu
        fba_final = fba
        #print(f"{bcolors.GREEN}mu = {mu:.7f}, status = {fba.status}{bcolors.RESET}")
    else:
        mu_max = mu
        #print(f"{bcolors.RED}mu = {mu:.7f}, status = {fba.status}{bcolors.RESET}")
        
    mu = float(mu_min + mu_max) / 2

In [52]:
class GAMS_settings:
    def __init__(self,
                 species_path='../model/RBA_species.txt',
                 rxn_path='../model/RBA_rxns.txt',
                 prosyn_path='../model/RBA_rxns_prosyn.txt',
                 nuc_trans_path='../model/RBA_nuc_translation.txt',
                 mito_trans_path='../model/RBA_mito_translation.txt',
                 uptake_path='../model/RBA_rxns_EXREV.txt',
                 media_path='../model/RBA_rxns_EXREV_YNB.txt',
                 sij_path='../model/RBA_sij.txt',
                 prolen_path='../model/RBA_proteinLength.txt',
                 kapp_path='../model/RBA_kapp.txt',
                 enz_cap_declares_path='../model/RBA_enzCapacityConstraints_declares.txt',
                 enz_cap_eqns_path='../model/RBA_enzCapacityConstraints_eqns.txt',
                 kribonuc='10.5*3600',
                 kribomito='10.5*3600',
                 mu='0'):
        self.species_path = species_path
        self.rxn_path = rxn_path
        self.prosyn_path = prosyn_path
        self.nuc_trans_path = nuc_trans_path
        self.mito_trans_path = mito_trans_path
        self.uptake_path = uptake_path
        self.media_path = media_path
        self.sij_path = sij_path
        self.prolen_path = prolen_path
        self.kapp_path = kapp_path
        self.enz_cap_declares_path = enz_cap_declares_path
        self.enz_cap_eqns_path = enz_cap_eqns_path
        self.kribonuc = kribonuc
        self.kribomito = kribomito
        self.mu = mu
    
    def export_to_txt_file(self, filepath):
        props = ['species_path', 'rxn_path', 'prosyn_path', 'nuc_trans_path',
                 'mito_trans_path', 'uptake_path', 'media_path', 'sij_path',
                 'prolen_path', 'kapp_path', 'enz_cap_declares_path',
                 'enz_cap_eqns_path', 'kribonuc', 'kribomito', 'mu']
        text = []
        for p in props:
            text.append('$setGlobal ' + p + ' ' + self.__dict__[p])
        
        with open(filepath, 'w') as f:
            f.write('\n'.join(text))
            
    def import_from_txt_file(self, filepath):
        with open(filepath) as f:
            text = f.read().split('\n')
        text = [i[11:] for i in text if i != '']
        for i in text:
            k,v = i.split(' ')
            self.__setattr__(k,v)

In [53]:
x = GAMS_settings()

In [54]:
x.mu

'0'

In [55]:
x.import_from_txt_file('./GAMS_settings.txt')

In [57]:
x.mu

'0.5'

In [20]:
x.export_to_txt_file('./GAMS_settings.txt')

In [12]:
x.__dict__

{'species_path': '../model/RBA_species.txt',
 'rxn_path': '../model/RBA_rxns.txt',
 'prosyn_path': '../model/RBA_rxns_prosyn.txt',
 'nuc_trans_path': '../model/RBA_nuc_translation.txt',
 'mito_trans_path': '../model/RBA_mito_translation.txt',
 'uptake_path': '../model/RBA_rxns_EXREV.txt',
 'media_path': '../model/RBA_rxns_EXREV_YNB.txt',
 'sij_path': '../model/RBA_sij.txt',
 'prolen_path': '../model/RBA_proteinLength.txt',
 'kapp_path': '../model/RBA_kapp.txt',
 'enz_cap_declares_path': '../model/RBA_enzCapacityConstraints_declares.txt',
 'enz_cap_eqns_path': '../model/RBA_enzCapacityConstraints_eqns.txt',
 'kribonuc': '10.5*3600',
 'kribomito': '10.5*3600',
 'mu': '0'}

In [14]:
x._ge

<method-wrapper '__setattr__' of GAMS_settings object at 0x7fe9183313d0>

In [5]:
class Rba_result:
    def __init__(self, growth_rate=None, raw_flux=None, rna_usage=0, proteome_usage=0,
                 proteome_allocation=None):
        self.growth_rate = growth_rate
        self.raw_flux = raw_flux
        self.rna_usage = rna_usage
        self.proteome_usage = proteome_usage
        self.proteome_fraction = proteome_allocation
        
    def calculate_rna_usage():
        None
        
    def calculate_proteome_usage():
        None
        
    def calculate_proteome_fraction():
        None

In [6]:
def get_GAMS_modelStat(txtfile='./runRBA.modelStat.txt'):
    with open(txtfile) as f:
        modelStat = f.read()
    modelStat = modelStat.replace('\n', '')
    modelStat = modelStat.replace(' ', '')
    modelStat = int(float(modelStat))
    if modelStat == 11:
        print('Licensing error')
    elif modelStat == 12:
        print('Error: Unknown cause')
    elif modelStat in [4,10,13,14,19]:
        stat = 'infeasible'
    elif modelStat == 1:
        stat = 'optimal'
    else:
        print('Feasible but not globally optimal')
        
    return stat

In [6]:
with open('./runRBA.gms', 'w') as f:
    f.write('\n'.join(gmstxt))

In [1]:
class Rba_result:
    def __init__(self, growth_rate=None, raw_flux=None, rna_usage=0, proteome_usage=0,
                 proteome_allocation=None):
        self.growth_rate = growth_rate
        self.raw_flux = raw_flux
        self.rna_usage = rna_usage
        self.proteome_usage = proteome_usage
        self.proteome_fraction = proteome_allocation
        
    def calculate_rna_usage():
        None
        
    def calculate_proteome_usage():
        None
        
    def calculate_proteome_fraction():
        None

In [13]:
class Rba_run:
    def __init__(self, input_files, mu=0):
        self.mu = mu
        
    

In [14]:
x = Rba_run()

AttributeError: 'Rba_run' object has no attribute 'files'

In [ ]:
x.fi